In [ ]:
from ipywidgets import Button, HBox, BoundedFloatText, FloatText
from IPython.display import display
import math

def wgs84_wm(lon_value, lat_value):
    r = 20037508.34
    x_value = lon_value / 180 * r
    y_value = math.log(math.tan(math.pi / 4 + lat_value * math.pi / 360)) * r / math.pi
    return x_value, y_value

def wm_wgs84(x_value, y_value):
    r = 20037508.34
    lon_value = x_value / r * 180
    lat_value = math.atan(math.exp(y_value * math.pi / r)) * 360 / math.pi - 90
    return lon_value, lat_value

def clear_lonlat(b):
    lon.value = 0
    lat.value = 0
    
def clear_xy(b):
    x.value = 0
    y.value = 0

def btn_84_wm_click(b):
    x.value, y.value = wgs84_wm(lon.value, lat.value)
    
def btn_wm_84_click(b):
    lon.value, lat.value = wm_wgs84(x.value, y.value)
    
lon = BoundedFloatText(
    description="经度:",
    value=0,
    min = -180.0,
    max = 180.0,
    step = 0.1
)
lat = BoundedFloatText(
    description="纬度:",
    value=0,
    min = -90.0,
    max = 90.0,
    step = 0.1
)

x = FloatText(
    description="x:",
    value=0
)
y = FloatText(
    description="y:",
    value=0
)

btn_84_wm = Button(description="WGS84转Web墨卡托")
btn_wm_84 = Button(description="Web墨卡托转WGS84")
btn_clear_lonlat = Button(description="经纬度清空")
btn_clear_xy = Button(description="xy清空")

btn_84_wm.on_click(btn_84_wm_click)
btn_wm_84.on_click(btn_wm_84_click)
btn_clear_lonlat.on_click(clear_lonlat)
btn_clear_xy.on_click(clear_xy)

hbox1 = HBox([lon, lat])
hbox2 = HBox([x, y])
hbox3 = HBox([btn_84_wm, btn_wm_84, btn_clear_lonlat, btn_clear_xy])
display(hbox1, hbox2, hbox3)

In [ ]:
from pyproj import CRS, Transformer
import math
wgs = CRS.from_proj4("+proj=longlat +datum=WGS84 +no_defs")
bj54 = CRS.from_proj4("+proj=longlat +ellps=krass +towgs84=15.8,-154.4,-82.3,0,0,0,0 +no_defs")
web_mercator = CRS.from_proj4("+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext  +no_defs")
transformer_wgs2bj54 = Transformer.from_crs(wgs,bj54,always_xy=True)
transformer_wgs2wm = Transformer.from_crs(wgs,web_mercator,always_xy=True)
transformer_bj542wm = Transformer.from_crs(bj54,web_mercator,always_xy=True)

point = (121, 31)

print(transformer_wgs2bj54.transform(point[0], point[1]) == point)

coord1 = transformer_wgs2wm.transform(point[0], point[1])
coord2 = transformer_bj542wm.transform(point[0], point[1])

def distance(coord1, coord2):
    dis = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
    return dis

print("两点间距: %.2f 米" % distance(coord1, coord2))